# Q-Learning

If we want to approximate the optimal controller in an MDP with unknown transition model, then one of the classical algorithms to use is Q-Learning.

## The MDP

The problem is provided by the <b>gym</b>-environment. 

The task is to reach the goal state <b>G</b>, starting from initial state <b>S</b>, without falling into Hole <b>H</b> and die miserably. That, means we have to reach the goal traversing the <b>F</b>  tiles. Remember that in the Frozenworld Problem the ground is slippery, meaning if you move right, you can also move upward or downward as well (the transition model is non-deterministic).

The map of the MDP is the following:

        S F F F
        F H F H
        F F F H
        H F F G

## Q-Learning

One of the early breakthroughs in reinforcement learning was the development of an off-policy TD control algorithm known as the Q-learning:

$Q(s,a) = Q(s,a) + \alpha \left[r + \gamma \max_{a'} Q(s', a') - Q(s,a)\right]$

## The Q-Learning Agent

In [1]:
import gym
import numpy as np
import random
import matplotlib as plt

class QL_Agent:
    def __init__(self, env, discount_factor, learning_rate, epsilon):
        self.g      = discount_factor
        self.lr     = learning_rate
        self.epsilon = epsilon
        self.terminals = [5,7,11,12,15]
        
        # Construct the Q table
        action_size = env.action_space.n
        state_size = env.observation_space.n
        self.Qtable = np.zeros((state_size, action_size))
        print("Q-Table")
        print("---------------------")
        print(self.Qtable)
        # MDP.p_a: uniform action probability (25% chance an action is selected)
        # MDP.p_s_: uniform next-state probability
        # MDP.P_ss_(s,a): set of all possible states when taking action a in state s
        # MDP.R(s): reward in state s
        # MDP.A: set of all possible actions,e.g. full action space
        # MDP.S_(s,a): next state when taking action a in state s 
        
        # 4x4 gridworld:
        #"S F F F",
        #"F H F H",
        #"F F F H",
        #"H F F G"

        #S: Start (constant starting position, reward=0)
        #F: Ice  (introduces stochastic action, reward=0)
        #H: Hole (ends episode, reward=0)
        #G: Goal (ends episode, reward=1)
        
    def action(self,s): # epsilon-Greedy action selection
        #LEFT = 0
        #DOWN = 1
        #RIGHT = 2
        #UP = 3
        
        # Draw a random number between 0 and 1
        exp_exp_tradeoff = np.random.uniform()
        
        # if random number greater that exploration probability, then pick the best action
        if exp_exp_tradeoff > self.epsilon:
            action = np.argmax(self.Qtable[s,:])
        # else select a random action
        else:
            action = env.action_space.sample()
        return(action)
    
    def update_QL(self,s,a,r,s_):
       # if s_ in self.terminals:
      #      self.Qtable[s_,:] = r
            
        self.Qtable[s, a] = self.Qtable[s, a] + \
        self.lr * (r + self.g * np.max(self.Qtable[s_, :]) - self.Qtable[s, a]) 

## Base Loop

In [2]:
env = gym.make("FrozenLake-v0")

# Exploration parameters
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.1            # Minimum exploration probability 
decay_rate = 0.0005             # Exponential decay rate for exploration prob
agentQL = QL_Agent(env, discount_factor=0.95,learning_rate=0.1, epsilon = max_epsilon)

total_episodes = 50000
rewards = np.zeros((total_episodes))
for i in range(total_episodes):
    total_rewards = 0
    s = env.reset()  # Initializes the Frozen Lake MDP
    while True:
        a = agentQL.action(s)  # Apply the epsilon-Greedy policy
        s_,r,done,_ = env.step(a)  # Observe the next state and the reward
        
        agentQL.update_QL(s,a,r,s_)  # Update the value function estimate using the latest transition
                           
        s = s_
        
        total_rewards += r
        
        # adjust exploration rate
        agentQL.epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*i) 
        if done:  # If episode terminated (i.e., the agent fell into a hole, discovered the goal state or max. number of steps were done)
            break 
            
    rewards[i] = total_rewards

print("Final Q-Table")
print("---------------------")
print(agentQL.Qtable)

Q-Table
---------------------
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Final Q-Table
---------------------
[[0.18839644 0.17838812 0.1655036  0.16261257]
 [0.11805545 0.1220741  0.10820042 0.1588671 ]
 [0.1700181  0.14765321 0.14724334 0.14658606]
 [0.08846954 0.09232435 0.05985638 0.13477897]
 [0.21563828 0.20047763 0.15178917 0.10428229]
 [0.         0.         0.         0.        ]
 [0.10654764 0.11917255 0.21084548 0.046918  ]
 [0.         0.         0.         0.        ]
 [0.19109406 0.25979893 0.20078452 0.27958926]
 [0.32040428 0.4260055  0.28111813 0.27185174]
 [0.47302199 0.33813754 0.26591254 0.17709788]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.29605103 0.43938688 0.57862265 0.46482726]
 [0.50793589 0.64300046 0.70509125 

## Evaluate our Q-Learning Agent after training

In [3]:
env.reset()
total_episodes = 300
rewards = np.zeros((total_episodes))
agentQL.epsilon = 0.0 # Greedy Policy

for i in range(total_episodes):
    total_rewards = 0
    s = env.reset()  # Initializes the Frozen Lake MDP
    while True:
        a = agentQL.action(s)  # Apply the epsilon-Greedy policy
        s_,r,done,_ = env.step(a)  # Observe the next state and the reward
        
#         agentQL.update_QL(s,a,r,s_)  # Update the value function estimate using the latest transition
                           
        s = s_
        
        total_rewards += r
        
        if done:  # If episode terminated (i.e., the agent fell into a hole, discovered the goal state or max. number of steps were done)
             # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            
            break 
            
    rewards[i] = total_rewards
    print("Evaluation Episode: " + str(i) + " --> Reward: " + str(rewards[i]))
    print("Mean Reward: " + str(np.sum(rewards / total_episodes)))

env.close()

  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 0 --> Reward: 1.0
Mean Reward: 0.0033333333333333335
  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 1 --> Reward: 0.0
Mean Reward: 0.0033333333333333335
  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 2 --> Reward: 1.0
Mean Reward: 0.006666666666666667
  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 3 --> Reward: 0.0
Mean Reward: 0.006666666666666667
  (Left)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 4 --> Reward: 0.0
Mean Reward: 0.006666666666666667
  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 5 --> Reward: 1.0
Mean Reward: 0.01
  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 6 --> Reward: 1.0
Mean Reward: 0.013333333333333334
  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 7 --> Reward: 0.0
Mean Reward: 0.013333333333333334
  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 8 --> Reward: 1.0
Mean Reward: 0.016666666666666666
  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 9 --> Reward: 1.0
Mean Reward: 0.02
  (Right)
S

SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 236 --> Reward: 0.0
Mean Reward: 0.48
  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 237 --> Reward: 0.0
Mean Reward: 0.48
  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 238 --> Reward: 0.0
Mean Reward: 0.48
  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 239 --> Reward: 0.0
Mean Reward: 0.48
  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 240 --> Reward: 1.0
Mean Reward: 0.48333333333333334
  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 241 --> Reward: 1.0
Mean Reward: 0.4866666666666667
  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 242 --> Reward: 1.0
Mean Reward: 0.49
  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 243 --> Reward: 1.0
Mean Reward: 0.49333333333333335
  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 244 --> Reward: 1.0
Mean Reward: 0.4966666666666667
  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 245 --> Reward: 1.0
Mean Reward: 0.5
  (Right)
SFFF
FHFH
FFFH
HFFG
Evaluation Episode: 246 --> Reward: 1.0
Mea